Before we begin, let's execute the below cell to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Exercise 6 [Optional]

###  Learning objectives
The **goal** of this lab is to:

- Learn how to profile an application using Nsight Systems and Compute (CUDA example)

We do not intend to cover:

- How to use CUDA programming model
- How to implement shared memory
- Optimization methods 

**NOTE: All the below screenshots are from A100 GPU.**

Please read the below notebooks before you start.

- Overview of Nsight profiler tools ([Nsight Systems](nsight_systems.ipynb) and [Nsight Compute](nsight_compute.ipynb))

In this lab, we will be porting the serial Jacobi code to the GPU using CUDA and obtaining the fast performance possible. To achieve this, we need to consider Amdahl's law (please see the section on [Amdahl's Law and Scaling](profiling_lab1.ipynb#amdahls)) and remove performance limitations as much as possible. Common performance limiters are:

- Serial portion of the code on the CPU (Read the section on [Amdahl's Law and Scaling](profiling_lab1.ipynb#amdahls))
- Memory movements (Device To/From Host)
- Latency as a result of launching GPU kernels
- Not enough work to hide instruction latency
- Not efficient memory access pattern
    - Uncoalesced memory accesses, lack of cache reuse, not using shared memory (Read more at [the NVIDIA Technical Blog](https://developer.nvidia.com/blog/boosting-application-performance-with-gpu-memory-prefetching/)) and [CUDA C Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/)
- Low arithmetic intensity (The ratio between compute work (FLOPs) and data movement (bytes))

### Step 0: Analyze the code

In this lab, we will be working on the Jacobi iteration code that iteratively converges to the correct value by computing new values at each point from the average of neighboring  points. 

<img src="images/jacobi_formula.png" width="90%">

**Understand and analyze** the code present at:
 
[Serial Code](../source_code/lab6/jacobi.cpp) 

To obtain the best performance from GPU and utilize the hardware, one should follow the cyclical process (analyze, parallelize, optimize), explained in the [Introduction section](introduction.ipynb#steps). Let's start by compiling the code and running it on the CPU (**Note:** error will be printed periodically as output).

In [ ]:
!cd ../source_code/lab6 && make

### Step 1: Instrument the code to find bottlenecks

Before we start the porting to the GPU, the first step is to identify the compute expensive part of the application. This step helps to find out the performance bottlenecks. One way would be to use the CPU walltime to measure different parts of the code, another method would be adding NVIDIA Tools Extension (NVTX). **Why are we using NVTX?** Please see the section on [Using NVIDIA Tools Extension (NVTX)](nsight_systems.ipynb#nvtx). We added the NVTX API to the code. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step1.cpp). (Reminder: we need to link it against the right runtime library (`libnvToolsExt.so`))

Now, let's compile the code and **profile** with `nsys` (Reminder: `--stats=true` shows the summary output that includes information about time spent in the various NVTX regions.) 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step1 && nsys profile --stats=true -o jacobi_step1 -f true ./jacobi_step1

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step1.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI  locally. 

The timeline of the application is as shown below (feel free to hover your mouse over each section to see more detail (for example, the thread state))

<img src="images/jacobi_1.png" width="90%">

### Step 2: Using Unified Memory

[Unified Memory(UM)](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/) is a single memory address space accessible from any processor in a system. It allows applications to allocate data that can be read or written from code running on either CPUs or GPUs. Now, to allocate Unified Memory, we replace calls to `malloc()` with calls to `cudaMallocManaged()` without making any other changes to the rest of the code. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step2.cpp). Now, let's compile and profile the code. 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step2 && nsys profile --stats=true -o jacobi_step2 -f true ./jacobi_step2

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step2.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI  locally. 

The timeline of the application is shown below.

<img src="images/jacobi_2.png" width="90%">

The application runs much longer than before and as you can see from the profiler report, there is a cost of initializing CUDA that can be high. In this example, this cost is much higher than the actual calculation time. 

<img src="images/jacobi_2_.png" width="90%">

### Step 3: Make the Problem Bigger

If we try to reduce the cost of computation and add fast kernels, it will still be slower than the CPU-only version (without UM).Similar to the previous lab, we simply need to make the problem bigger by either adding more particles/elements or increasing the iteration count. In this example, we increase the grid points `N` which achieves a finer spatial resolution and is more accurate. This will make the Jacobi relaxation step a big chunk of the total application runtime. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step3.cpp). Now, let's compile and profile the code. 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step3 && nsys profile --stats=true -o jacobi_step3 -f true ./jacobi_step3

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step3.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI  locally. 

With a close look at the timeline view, we can see that the Jacobi relaxation step is now a big portion of the total application runtime.

<img src="images/jacobi_3.png" width="90%">

You can also see the most expensive portion of the application from the profiler report. Right click on the NVTX row from the timline view, then choose "Show in Events View". Then, from the box at the bottom, you can sort all the ranges by duration. Now, you can see the `jacobi_step()` (Jacobi relaxation step) is the most compute expensive part of the code.

<img src="images/jacobi_3_event.png" width="90%">

### Step 4: Port the `jacobi_step` kernel to GPU using CUDA

We convert the `jacobi_step()` to a CUDA kernel, parallelising over the inner and outer loop using a 2D threadblock of size `32x32`. Except for the `error`, each part can be updated independently `atomicadd()` to perform that. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step4.cpp). Now, let's compile and profile the code. 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step4 && nsys profile --stats=true -o jacobi_step4 -f true ./jacobi_step4

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step4.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI locally. Let's check the NVTX ranges in the "Events View". We can see that the "Jacobi Step" is significantly faster but the "Swap data" is slower than before.

<img src="images/jacobi_4_.png" width="40%">

If we look at the CUDA row, and zoom in, we see all those gaps that are marked as "Swap data" regions on the NVTX row. There is a lot of data movement from host to device in `jacobi_step()` kernel and device to host in the `swap_data()` function (due to the Unified Memory usage).

<img src="images/jacobi_4_1.png" width="80%">

### Step 5: Port the `swap_data` kernel to GPU using CUDA 

We port the `swap_data()` function to the GPU using CUDA to do most of the computation on the device and reduce the data movement by keeping most of the data on the GPU. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step5.cpp). Now, let's compile and profile the code. 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step5 && nsys profile --stats=true -o jacobi_step5 -f true ./jacobi_step5

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step5.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI  locally. Let's check the NVTX ranges in the "Events View". We can see that the application is much faster now ("Swap data" is now faster than before). 

<img src="images/jacobi_5_0.png" width="40%">

The timeline of the application is shown below. 

<img src="images/jacobi_5_1.png" width="80%">

Let's look at the CUDA row, it is clear that most of the application runtime is now spent in kernels and we also reduced the data movement (green rectangles). Let's dig deeper and analyze the kernels in [Nsight Compute](nsight_compute.ipynb) to see if there are any other optimizations we can apply.


### Step 6: Analyze and improve the Kernels

In this section, we want to find out if the kernel is compute-bound, memory-bound, or latency-bound. To ensure the kernel is not limited by latency, we need to expose enough work to keep a large number of threads running on the GPU. In this example, `N` is equal to 2048, which means the problem size is `2048 x 2048` and since the order of magnitude number of threads a modern GPU can run simultaneously is O(100k), then we probably have enough work to keep the device busy. 

In the case of memory bound or compute bound, we need to think of arithmetic intensity which is the ratio between compute work (FLOPs) and data movement (bytes)) where FLOPS is Floating Point Operations per second. If this ratio is of order 10, then it would be compute bound. 

```cpp
f[IDX(i,j)] = 0.25f * (f_old[IDX(i+1,j)] + f_old[IDX(i-1,j)] +
                      f_old[IDX(i,j+1)] + f_old[IDX(i,j-1)]);
```

The `jacobi_step()` kernel computes four floating point operations (three adds and one multiply) and moves four words (each 4 bytes for single precision floating point), so the arithmetic intensity is `4 (FLOPs) / 16 (bytes) = 0.25 FLOPs / byte` and it seems to be in the memory bandwidth region.

Now, let's profile the kernel with Nsight Compute to verify our hypotheses. Make sure to read the [Nsight Compute](nsight_compute.ipynb) notebook before you continue. 

We only profile one invocation of the kernel since it has similar performance characteristics, and to allow the device to warmp up we skip the first few.

In [ ]:
!cd ../source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi -f -o jacobi_step5_0 --set full ./jacobi_step5

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step5_0.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI locally. 

The Speed of Light Throughput section gives a high-level overview of the throughput for compute and memory resources of the GPU For each unit. Based on this information we can find the performance limiters and catagorize them into four possible combinations:

- Compute Bound: SM>50% & Mem<50%
- Bandwidth Bound: SM<50% & Mem>50%
- Latency Bound: SM<50% & Mem<50%
- Compute and Bandwidth Bound : SM>50% & Mem>50%

According to the *GPU Speed of Light Throughput* section (see screenshot below), the kernel exhibits low compute throughput and memory bandwidth utilization relative to the peak performance of the device it was run on and since Achieved compute throughput and/or memory bandwidth below 60.0% of peak, the kernel is latency bound. The tool suggests looking at the *Scheduler Statistics* and *Warp State Statistics* for potential reasons. 

<img src="images/jacobi_5_sol.png" width="80%">

The *Scheduler Statistics* shows the summary of the activity of the schedulers issuing instructions. Every scheduler is capable of issuing one instruction per cycle, but for this kernel, each scheduler only issues an instruction every ~1287 cycles. This causes the hardware resources underutilized and leads to less optimal performance. 

<img src="images/jacobi_5_schedule.png" width="80%">

Let's check the *Warp State Statistics* section  which presents an analysis  of the states in which all warps spent cycles during the kernel execution. The warp states describe a warp's readiness or inability to issue its next instruction. As seen in the screenshot below, the most important stall reason is *LG (local/global) Throttle*. This indicates extremely frequent memory instructions, according to the guided analysis rule (hover your mouse over each to see the description).

<img src="images/jacobi_5_warp.png" width="80%">

When we check the *Source Counters* section for the top stall locations in your source based on sampling data, we can see the kernel has uncoalesced global accesses (if you click on the links you can go to the line where these occur). 

<img src="images/jacobi_5_source.png" width="80%">

Last, looking at the *Occupancy* section, we achieved 71% occupancy which is the ratio of the number of active warps per multiprocessor to the maximum number of possible active warps. In other words, we are giving enough work to the device to keep it busy. So, the low memory throughput is a result of poor memory access patterns.

<img src="images/jacobi_5_occ.png" width="80%">

To conclude, the reason for low memory throughput is poor memory access patterns which means the memory layout of our arrays does not map well with the threads (Read more at [Analysis-Driven Optimization: Analyzing and Improving Performance with NVIDIA Nsight Compute](https://developer.nvidia.com/blog/analysis-driven-optimization-analyzing-and-improving-performance-with-nvidia-nsight-compute-part-2/)). Now, let's profile the `swap_data()` kernel with Nsight Compute.

In [ ]:
!cd ../source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:swap_data -f -o jacobi_step5_1 --set full ./jacobi_step5

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step5_1.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI locally. 

According to the *GPU Speed of Light Throughput* section (see screenshot below), the memory is more heavily utilized than compute and we need to look at the *Memory Workload Analysis* to identify the L2 bottleneck. 

<img src="images/jacobi_swap_sol.png" width="80%">

The *Memory Workload Analysis* section tells us that the kernel has a poor memory access pattern. Rather than accessing 4 bytes per thread per memory request, there are `32x32 = 1023` bytes of cache data transfers per request which is 8 times more sector loads.

<img src="images/jacobi_swap_mem.png" width="80%">

Let's compare the indexing scheme we used to the threading scheme. In a 2D threadblock, the `x` dimension is the contiguous dimension and the `y` dimension is the strided dimension. So, in a 32x32 thread block, each warp comprises of 32 threads in the `x` dimension and the `threadIdx.y` counts/enumerates 32 warps. To fix the uncoalessed memory access, we change `#define IDX(i, j) ((j) + (i) * N)` to `#define IDX(i, j) ((i) + (j) * N)`. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step6.cpp). 

Now, let's compile and profile the code. 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step6 && nsys profile --stats=true -o jacobi_step6 -f true ./jacobi_step6

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step6.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI locally. We can clearly see we made some overall improvements and reduced the total execution time.

Let's profile the `swap_data` kernel with Nsight Compute to see if we fixed the global memory access pattern.

In [ ]:
!cd ../source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:swap_data -f -o jacobi_step6_1 --set full ./jacobi_step6

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step6_1.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI  locally. When comparing the `swap_data` kernel with the previous step, we can see that we fixed the memory access pattern and improved the overall performance.

<img src="images/jacobi_6_swap_base.png" width="80%">

We are now latency bound and if you look at the *Source Counters* section, we no longer have any noncoalesced global accesses. Now, let's profile the `jacobi_step` kernel with Nsight Compute.

In [ ]:
!cd ../source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi -f -o jacobi_step6_0 --set full --import-source 1 ./jacobi_step6

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step6_0.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI locally. Comparing to the previous version of the code, the `jacobi_step` kernel did achieve a substantial improvement.

<img src="images/jacobi_6_jacobi_base.png" width="80%">

Looking at the *Scheduler Statistics*, we see 99.9% of the cycles have no warps eligible to issue work. On cycles with no eligible warps, the issue slot is skipped and no instruction is issued. Having many skipped issue slots indicates poor latency hiding. 

<img src="images/jacobi_6_jacobi_schedule.png" width="80%">

The reason for this is the atomic update to the error counter inside the `jacobi_step()` kernel (`atomicAdd(error, df * df)`). If multiple threads write to the same location at the same time, they will serialize and stall. We can refactor the kernel in a way to use a more efficient reduction scheme that uses fewer overall atomics. Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step7.cpp). Now, let's compile and profile the code. 

In [ ]:
!cd ../source_code/lab6 && make jacobi_step7 && nsys profile --stats=true -o jacobi_step7 -f true ./jacobi_step7 

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step7.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI locally. 

We managed to reduce the total execution time enormously. Let's profile the `jacobi_step()` kernel with Nsight Compute to investigate more.

In [ ]:
!cd ../source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi -f -o jacobi_step7_0 --set full --import-source 1 ./jacobi_step7

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step7_0.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI locally. 

<img src="images/jacobi_7_sol.png" width="80%">

The *GPU Speed of Light* section shows we increase both Compute and Memory utilization compared to the previous section. Moreover, the roofline charts shows an improvement in performance and an increase in arithmetic intensity.

<img src="images/jacobi_7_roof.png" width="80%">

Comparing the `jacobi_step` and `swap_data` kernel, one has more DRAM throughput than the `jacobi_step` kernel. 

<img src="images/jacobi_7_base.png" width="80%">

### Step 7: Shared Memory
Although the code runs faster, there are still uncoalesced accesses that we need to address. The issue with the stencil operations is the cache reuse and as the array (problem size) grows, due to the stride between row `j` and row `j+1` , there will be less cache reuse. To solve this issue, we can read the data from the shared memory which is closer to the GPU cores. This will result in accessing the DRAM less. We implement the simplest version for ease but this might not be the best solution. To learn more on how to understand and optimize Shared Memory accesses using Nsight Compute, please watch the [GTC talk](https://www.nvidia.com/en-us/on-demand/session/gtcspring22-s41723/) on the same topic.

Feel free to inspect the [Jacobi Code](../source_code/lab6/jacobi_step8.cpp). Now, let's compile and profile the code.

In [ ]:
!cd ../source_code/lab6 && make jacobi_step8 && nsys profile --stats=true -o jacobi_step8 -f true ./jacobi_step8

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step8.nsys-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight System UI locally. 

Let's profile the kernel with Nsight Compute and inspect it further.

In [ ]:
!cd ../source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi --export jacobi_step8_0 --force-overwrite --set full ./jacobi_step8

Download and save the report file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../source_code/lab6/jacobi_step8_0.ncu-rep) and choosing <mark>Save Link As</mark>. Once done open the report via the Nsight Compute UI locally. 

<img src="images/jacobi_8_sol.png" width="80%">

We achieved better compute and memory utilization compared to the previous section and memory access patterns is fully coalesced.


<img src="images/jacobi_8_source.png" width="80%">

The Nsight Systems profiler report shows how fast the kernels are. However, take to account that you might have to increase the problem size again as once again the device warmup might be a performance limiter.

<img src="images/jacobi_8_nsight.png" width="80%">

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommended you go to your browser's File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f _profiler_files.zip
zip -r _profiler_files.zip *

**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../_profiler_files.zip) and choosing <mark>Save Link As</mark>. 

-----

# <p style="text-align:center; border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../_start_profiling.ipynb>HOME</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span style="float:center"> <a href=profiling_lab6.ipynb>NEXT</a></span> </p>

-----

# Links and Resources


[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://docs.nvidia.com/nsight-compute/index.html)


**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System's latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).